In [1]:
pip install ipynb --upgrade

In [1]:
from ipynb.fs.full.rngs import plantSeeds, selectStream, random
from ipynb.fs.full.rvms import cdfNormal,idfNormal
from colorama import Fore, Style, Back
from math import log,exp,fabs, sqrt

seed = 123456798
#seed = 365676778
#seed = 222222222

#centers = [0,2,3,5,2]
#centers = [0,2,6,22,7]
centers = [0,5,11,50,12]

NODES=4
TINY = 1.0e-10
SQRT2PI = 2.506628274631  # #/* sqrt(2 * pi) */

LAMBDA1LOW=0.004067*1.1
LAMBDA1MID=0.015373*1.1
LAMBDA1HIGH=0.073208*1.1

LAMBDA2LOW=0.000208*1.1
LAMBDA2MID=0.000667*1.1
LAMBDA2HIGH=0.008271*1.1

MAXIMUM=10000000

CASS=1
KITCHEN=2
TABLE=3
DELIVERY=4

START =   0.0                    # initial (open the door)        */
STOP  =   75600.0                # secondi delle 21 ore del mcDonald */
arrivalTemp = [START for x in range(NODES+1)]
delivery=[]

global flag
flag=0


class TypeNode:
    id= None 
    ev= None
    servers=0.0 
    index=None  
    sum=0.0 
    number= 0.0 
    area=0.0 
    flow=None 
    lamb=0.0  
    
    
    def __init__(self, id_node):
        self.id = id_node
        self.ev = Event()
        self.index = AccumSum()
        self.flow= Time()
        

class Event:
    t = 0.0 #next event time
    x =None  #event status, 0 or 1
    last= 0.0
    dv = 0

class Time:
    current = None          # current time                       */
    next = None             # next (most imminent) event time    */

class AccumSum:
                          # accumulated sums of                */
    service =0.0          #   service times                    */
    served = 0.0           #   number served                    */

class Data:
    
    node=None
    event=None
    verify=0

def Exponential(m):
# ---------------------------------------------------
# * generate an Exponential random variate, use m > 0.0 
# * ---------------------------------------------------
# */
  return (-m * log(1.0 - random()))


def Normal(m, s):
    # ========================================================================
    # Returns a normal (Gaussian) distributed real number.
    # NOTE: use s > 0.0
    #
    # Uses a very accurate approximation of the normal idf due to Odeh & Evans,
    # J. Applied Statistics, 1974, vol 23, pp 96-97.
    # ========================================================================
    #
    p0 = 0.322232431088
    q0 = 0.099348462606
    p1 = 1.0
    q1 = 0.588581570495
    p2 = 0.342242088547
    q2 = 0.531103462366
    p3 = 0.204231210245e-1
    q3 = 0.103537752850
    p4 = 0.453642210148e-4
    q4 = 0.385607006340e-2

    u = random()
    if (u < 0.5):
        t = sqrt(-2.0 * log(u))
    else:
        t = sqrt(-2.0 * log(1.0 - u))

    p = p0 + t * (p1 + t * (p2 + t * (p3 + t * p4)))
    q = q0 + t * (q1 + t * (q2 + t * (q3 + t * q4)))

    if (u < 0.5):
        z = (p / q) - t
    else:
        z = t - (p / q)

    return (m + s * z)



def TruncatedNormal(m, s, a, b):
    # ---------------------------------------------------
# * generate a Truncated Normal random variate, use m,s,a,b > 0.0,
# * ---------------------------------------------------
# */
    alpha = cdfNormal(m, s, a)
    beta = 1.0 - cdfNormal(m, s, b)
    u = Uniform(alpha, 1.0 - beta)
    return idfNormal(m, s, u)

def Uniform(a,b):  
# --------------------------------------------
# * generate a Uniform random variate, use a < b 
# * --------------------------------------------
# */
  return (a + (b - a) * random())  


def GetArrival(m, node):
# ----------------------------------------------------
# * generate the next arrival time, with rate variable
# * --------------------------------------------------
# */ 
    global arrivalTemp
    
    if node==CASS:
        selectStream(0)
        arrivalTemp[node] += Exponential(1/m)
        return (arrivalTemp[node])
    elif node==KITCHEN:
        selectStream(1)
        arrivalTemp[node] += Exponential(1/m)
        return (arrivalTemp[node])
    
def GetService(i):
# --------------------------------------------
# * generate the next service time with rate 1/6
# * --------------------------------------------
# */ 
     
    if i == CASS:
        selectStream(2)
        return (TruncatedNormal(45,4,10,80))
    elif i == KITCHEN:
        selectStream(3)
        return (TruncatedNormal(112,5,60,164))
    elif i == TABLE:
        selectStream(4)
        return (TruncatedNormal(600,20,200,1600))
    elif i == DELIVERY:
        selectStream(5)
        return (TruncatedNormal(1200,40,800,2000))
        


def NextNode(nodes):
# -------------------------------------
# * return the index of the next node 
# * -----------------------------------
# */
    time_events=[]
    
    data=Data()
    
    for i in range (1, len(nodes)):
        for s in range(nodes[i].servers + 1):
            time_events.append(nodes[i].ev[s].t)
                
    time_events = sorted(time_events, key=lambda x: (x == 0.0, x))
    if time_events[0] ==MAXIMUM:
        data.verify=1
        
    for i in range(1, len(nodes)):
        for j in range(nodes[i].servers + 1):
            if time_events[0] == nodes[i].ev[j].t:
                data.node=i
                data.event=j
                return (data)
    
    
def lambdaFunc(nodes, i):
  
    a=0.9247365
    
    if  nodes[i].flow.current < 7200 or 25200 <=  nodes[i].flow.current < 32400 or 68400 <= nodes[i].flow.current < STOP:
        n1=LAMBDA1LOW
        n2=LAMBDA2LOW
    elif  39600 <=  nodes[i].flow.current < 50400:
        n1=LAMBDA1HIGH
        n2=LAMBDA2HIGH
    else:
        n1=LAMBDA1MID
        n2=LAMBDA2MID
    
    
    if i == CASS:
        nodes[i].lamb=n1
            
    elif i == KITCHEN:
        nodes[i].lamb=n2
            

def FindOne(node):
# -----------------------------------------------------
# * return the index of the available server idle longest
# * -----------------------------------------------------
# */
    i = 1

    while (node.ev[i].x == 1):       # find the index of the first available */
        i += 1                        # (idle) server                         */
    #EndWhile
    s = i
    
    while (i < node.servers):         # now, check the others to find which   */ 
        i += 1                        # has been idle longest                 */
        if ((node.ev[i].x == 0) and (node.ev[i].last <= node.ev[s].last)):
            s = i
   #EndWhile 
    return (s)


def forwardFunc(node):
    
    global flag
        
    if node==CASS:
        forwardNode(CASS,KITCHEN)    
                
    if node==KITCHEN:  
        if nodes[node].ev[e].dv==1:
            forwardNode(KITCHEN,DELIVERY)
                
        else:
            forwardNode(KITCHEN,TABLE)
        
    
        if flag==1:
            nodes[node].ev[e].dv=1 
            flag=0
                
    if node==TABLE or node==DELIVERY:
        nodes[0].number -= 1

        
def forwardNode(startNode,forwardNode):
        nodes[forwardNode].flow.next    = nodes[startNode].ev[e].last
        nodes[forwardNode].area     += ( nodes[forwardNode].flow.next - nodes[forwardNode].flow.current) * nodes[forwardNode].number  # update integral  */
        nodes[forwardNode].flow.current = nodes[forwardNode].flow.next 
        nodes[forwardNode].number += 1
        if startNode == KITCHEN and forwardNode== DELIVERY:
            nodes[startNode].ev[e].dv=0
        if (nodes[forwardNode].number <= nodes[forwardNode].servers):
            service  = GetService(forwardNode)
            s = FindOne(nodes[forwardNode])
            nodes[forwardNode].index[s].service += service
            nodes[forwardNode].index[s].served += 1                                          

            nodes[forwardNode].ev[s].t      =  nodes[startNode].flow.current + service
            nodes[forwardNode].ev[s].x      = 1
        #EndIf

                
def exitFunc(node):
            
    exit=0
                                         
    if nodes[node].number-nodes[node].servers>nodes[node].servers*4:
        exit=0.005*(nodes[node].number-nodes[node].servers)/nodes[node].servers
        if exit>0.9:
            exit=0.9
                       
    if exit!=0:
        if random()< exit :
            nodes[node].number -= 1
            nodes[0].number -= 1

############################Main Program###################################


global current_for_update

current_for_update=0
node=0
num1=0.86733333
num2=0.13286667
nodes = [TypeNode(i) for i in range(NODES+1)]
t = [0.0 for i in range(NODES+1)]
array = [0.0 for i in range(NODES+1)]

for i in range(NODES+1):
    nodes[i].servers=centers[i]


for i in range(NODES +1):
    nodes[i].ev = [Event() for j in range(nodes[i].servers + 1)]
    nodes[i].index=[AccumSum() for j in range(nodes[i].servers + 1)]
    nodes[i].flow=Time()
              
plantSeeds(seed)

nodes[0].ev[0].x=1

for i in range(1, NODES+1):
    nodes[i].flow.current = START
    lambdaFunc(nodes, i)
    if i==CASS or i==KITCHEN:
        nodes[i].ev[0].t   = GetArrival(nodes[i].lamb,i)
    nodes[i].ev[0].x   = 1

for i in range(1, NODES +1):
    for s in range(1,nodes[i].servers + 1):
        nodes[i].ev[s].t     = START          # this value is arbitrary because */
        nodes[i].ev[s].x     = 0              # all servers are initially idle  */
        nodes[i].index[s].service = 0.0
        nodes[i].index[s].served  = 0
       
    
while ((nodes[0].ev[0].x != 0) or (nodes[0].number != 0)):
      
    elem      = NextNode(nodes)                                      # next event index */
     
    node=elem.node
    e=elem.event
    
    if elem.verify==1:
        nodes[node].number-=1
        nodes[0].number-=1
  
    nodes[node].flow.next    = nodes[node].ev[e].t                                       # next event time  */ 
    
    if nodes[node].flow.current != MAXIMUM and nodes[node].flow.next !=MAXIMUM :
        nodes[node].area     += ( nodes[node].flow.next - nodes[node].flow.current) * nodes[node].number  # update integral  */
        nodes[node].flow.current = nodes[node].flow.next                                              # advance the clock*/
        
    
    if nodes[3].ev[0].x==0 and nodes[4].ev[0].x==0:
        nodes[0].ev[0].x    = 0
        
    if nodes[1].ev[0].x==0 and nodes[2].ev[0].x==0:
            for j in range(3,5):
                if nodes[j].number==0:
                    nodes[j].ev[0].x=0
            
    
  


    if (e == 0):                                                                 # process an arrival*/     
        
        lambdaFunc(nodes, node)           
        nodes[node].number += 1
        nodes[0].number += 1
        nodes[node].ev[e].last= nodes[node].ev[0].t
            
        
        if node ==CASS:
            
            exitFunc(node)
                
                    
        if node==CASS or node ==KITCHEN:
            nodes[node].ev[0].t        = GetArrival(nodes[node].lamb, node)
            
        if node ==KITCHEN and nodes[node].number > nodes[node].servers: 
            delivery.append(nodes[node].number - nodes[node].servers)            
        
        if (nodes[node].ev[0].t > STOP):
            current_for_update=nodes[node].ev[0].t      ########### nei nodi 3 4 da mettere
            nodes[node].ev[0].x    = 0
            nodes[node].ev[0].t    =MAXIMUM
                            
        #EndIf
        if (nodes[node].number <= nodes[node].servers and nodes[node].number !=0 ):
            service  = GetService(node)
            s = FindOne(nodes[node])
            if node==KITCHEN:
                nodes[node].ev[s].dv=1
            nodes[node].index[s].service += service
            nodes[node].index[s].served += 1                                          ############
            
            nodes[node].ev[s].t      =  nodes[node].flow.current + service
            nodes[node].ev[s].x      = 1
            #EndIf
       #EndIf
    else:                                          # process a departure */
 
        nodes[node].sum += 1                                     # from server s       */     ########

        nodes[node].number -= 1
        
        nodes[node].ev[e].last= nodes[node].ev[e].t 
        nodes[node].ev[e].t= MAXIMUM

        s = e                       
        if (nodes[node].number >= nodes[node].servers):
            if node ==2 and len(delivery)!=0:
                for j in range(len(delivery)):
                    delivery[j]=delivery[j]-1
                if delivery[0]==0:
                    delivery.pop(0)
                    flag=1
                   
            service   = GetService(node)
            nodes[node].index[s].service += service
            nodes[node].index[s].served += 1                                           #############
            nodes[node].ev[s].t      =  nodes[node].flow.current + service
        else:
            nodes[node].ev[s].x      = 0
            
        forwardFunc(node)
        
        #EndElse
  
 #EndWhile              
                               
io = 0


for i in range(1, NODES +1):
    average=0
    io = 0
    for s in range(1,nodes[i].servers +1):
        io+=nodes[i].index[s].served
    print("\nfor {0:1f} jobs the service node statistics are:\n".format(nodes[i].sum))
    print("  avg interarrivals .. = {0:6.7f}".format(current_for_update /nodes[i].sum))
    print("  avg wait ........... = {0:6.7f}".format(nodes[i].area /nodes[i].sum))
    print("  avg # in node ...... = {0:6.7f}".format(nodes[i].area / current_for_update))
    
    for s in range(1, nodes[i].servers+1):            # adjust area to calculate */ 
        nodes[i].area -= nodes[i].index[s].service   # averages for the queue   */
        if nodes[i].area <0:
            nodes[i].area=0
        average += nodes[i].index[s].service / nodes[i].index[s].served
        
        
    
    print("  avg delay .......... = {0:6.7f}".format(nodes[i].area /nodes[i].sum))
    c=nodes[i].area /nodes[i].sum
    print("  avg # in queue ..... = {0:6.7f}".format(nodes[i].area /  current_for_update))
    print("  avg service time.....= {0:6.7f}".format(average/nodes[i].servers)) 
    d=average/nodes[i].servers
    w=c+d
    array[i]=w


final=(array[1]+array[2]+array[3])*num1 +(array[2]+array[4])*num2

    
    
print("\n\navg wait system:   ", final)    
print("\n\nthe server statistics are:\n")
print("    server     utilization     avg service        share\n")


for i in range(1, NODES +1):    
    for s in range(1,nodes[i].servers +1):
        if nodes[i].index[s].service /  current_for_update >= 0.9:
                print(Fore.RED + "\n utilization asymptotic to 1")
                print(Fore.BLACK)
                if nodes[i].index[s].service /  current_for_update >= 1:
                    nodes[i].index[s].service = current_for_update
        print("{0:8d} {1:14.7f} {2:15.7f} {3:15.7f}".format(s, nodes[i].index[s].service /  current_for_update,
                                                            nodes[i].index[s].service / nodes[i].index[s].served,
                                                            float(nodes[i].index[s].served) / nodes[i].sum))



for 1603.000000 jobs the service node statistics are:

  avg interarrivals .. = 47.7235582
  avg wait ........... = 49.2976248
  avg # in node ...... = 1.0329830
  avg delay .......... = 4.2131812
  avg # in queue ..... = 0.0882830
  avg service time.....= 45.0841737

for 1733.000000 jobs the service node statistics are:

  avg interarrivals .. = 44.1436028
  avg wait ........... = 120.1490564
  avg # in node ...... = 2.7217773
  avg delay .......... = 8.3241547
  avg # in queue ..... = 0.1885699
  avg service time.....= 111.8253600

for 1603.000000 jobs the service node statistics are:

  avg interarrivals .. = 47.7235582
  avg wait ........... = 609.5815782
  avg # in node ...... = 12.7731796
  avg delay .......... = 10.4140779
  avg # in queue ..... = 0.2182167
  avg service time.....= 599.1821819

for 130.000000 jobs the service node statistics are:

  avg interarrivals .. = 588.4681825
  avg wait ........... = 1405.2300031
  avg # in node ...... = 2.3879456
  avg delay ..........